# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [47]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [48]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [49]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True, parser='auto')
X = data.data.to_numpy()
y = data.target.astype(float).to_numpy() # Assicura che il target sia float per la regressione

# Definizione della pipeline
def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    # PCA (eventuale standardizzazione + PCA)
    if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
            scalerPCA = StandardScaler()
            X_train = scalerPCA.fit_transform(X_train)
            X_val = scalerPCA.transform(X_val)

        pca = PCA(n_components=hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

    # Standardizzazione finale
    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunge il termine costante ai dati
    train_ones = np.ones(X_train.shape[0])
    validation_ones = np.ones(X_val.shape[0])
    X_train = np.c_[X_train, train_ones]
    X_val = np.c_[X_val, validation_ones]

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        reg_lambda = hyperparams['reg_lambda']  # assegno a lambda il valore desiderato
        # Applico la formula con la regolarizzazione
        RL = np.linalg.inv((X_train.T @ X_train) + reg_lambda * np.eye(X_train.shape[1])) @ X_train.T @ y_train
    else:
        RL = np.linalg.inv((X_train.T @ X_train)) @ X_train.T @ y_train

    # Calcolo predizioni
    y_pred_train = X_train @ RL
    y_pred_val = X_val @ RL

    # Calcola il MAE
    mae_train = np.mean(np.abs(y_train - y_pred_train))
    mae_val = np.mean(np.abs(y_val - y_pred_val))

    return RL, mae_train, mae_val

# Dividi il dataset in training e test set
n = X.shape[0]
num_train = int(0.6 * n)
num_val = int(0.2 * n)

# Dividi il training set in training set effettivo e validation set
X_train = X[:num_train]
y_train = y[:num_train]
X_val = X[num_train:num_train + num_val]
y_val = y[num_train:num_train + num_val]
X_test = X[num_train + num_val:]
y_test = y[num_train + num_val:]

# Trova la configurazione di iperparametri migliore
def trova_migliore(X_train, y_train, X_val, y_val, configs):
    migliore_config = None
    mae_val_migliore = float("inf")
    mae_train_migliore = None
    pesi_migliori = None

    for config in configs:
        RL, mae_train, mae_val = pipeline(X_train, y_train, X_val, y_val, config)
        if mae_val < mae_val_migliore:
            mae_val_migliore = mae_val
            mae_train_migliore = mae_train
            migliore_config = config
            pesi_migliori = RL

    return migliore_config, mae_train_migliore, mae_val_migliore, pesi_migliori

migliore_config, mae_train_migliore, mae_val_migliore, pesi_migliori = trova_migliore(X_train, y_train, X_val, y_val, configs)

# Riallena il modello sul training set completo
X_train_val = np.concatenate([X_train, X_val])
y_train_val = np.concatenate([y_train, y_val])
RL, mae_train_val_migliore, _ = pipeline(X_train_val, y_train_val, X_test, y_test, migliore_config)

# Calcola il MAE sul test set
def calcola_mae_test(X_train, y_train, X_test, y_test, hyperparams):
    if hyperparams['use_pca']:
        if hyperparams['pca_standardize']:
            scalerPCA = StandardScaler()
            X_train = scalerPCA.fit_transform(X_train)
            X_test = scalerPCA.transform(X_test)

        pca = PCA(n_components=hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)

    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    X_train = np.c_[X_train, np.ones(X_train.shape[0])]
    X_test = np.c_[X_test, np.ones(X_test.shape[0])]

    if hyperparams['use_regularization']:
        reg_lambda = hyperparams['reg_lambda']  # assegno a lambda il valore desiderato
        # Applico la formula con la regolarizzazione
        RL = np.linalg.inv((X_train.T @ X_train) + reg_lambda * np.eye(X_train.shape[1])) @ X_train.T @ y_train
    else:
        RL = np.linalg.inv((X_train.T @ X_train)) @ X_train.T @ y_train

    y_pred_test = X_test @ RL
    mae_test = np.mean(np.abs(y_test - y_pred_test))
    return mae_test

mae_test_migliore = calcola_mae_test(X_train_val, y_train_val, X_test, y_test, migliore_config)

# Stampa  risultati
print("\n=== RISULTATI FINALI ===")
print("Configurazione migliore trovata:")
print(migliore_config)
print(f"\nMiglior MAE sul Validation Set: {mae_val_migliore:f}")
print(f"Miglior MAE sul Training Set: {mae_train_val_migliore:f}")
print(f"Miglior MAE sul Test Set: {mae_test_migliore:f}")


=== RISULTATI FINALI ===
Configurazione migliore trovata:
{'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}

Miglior MAE sul Validation Set: 0.587261
Miglior MAE sul Training Set: 0.594274
Miglior MAE sul Test Set: 0.548441
